In [1]:
# wd = '/Users/marcinsawinski/Documents/GitHub/factue-task2'
wd = '/mnt/openfact/users/msawinski/factue-task2'
import sys, os
os.chdir(wd)

In [2]:
import pandas as pd
df = pd.read_parquet('data/llm_output/persuasion/detect/LLAMA_32_3B/attack/Name_Calling-Labeling_v001/train/train-pl/batch_0000.parquet')
df

,filename,start,end,text_lang,text,label_bin,labels_multi,output_id,prompt_id,job,...,extra_content,is_valid,raw,description,verdict,illegal_value,think_content,extra_properties,pred,gold
0,pl_current_affairs_09_01_2025_n03.txt,0,87,PL,Wicemarszałek Włodzimierz Czarzasty:\nDziękuję...,False,[],LLAMA_32_3B/attack/Name_Calling-Labeling_v001,attack/Name_Calling-Labeling_v001,persuasion,...,"[None, None, None]","[True, True, True]","[{""Description"": ""Technique used: No — Name Ca...",[Technique used: No — Name Calling / Labeling ...,"[0, 0, 0]","[None, None, None]","[None, None, None]","[None, None, None]",0,0
1,pl_current_affairs_09_01_2025_n03.txt,89,261,PL,Poseł Anna Gembicka:\nBardzo dziękuję.\nPanie ...,True,"[Questioning_the_Reputation, Name_Calling-Labe...",LLAMA_32_3B/attack/Name_Calling-Labeling_v001,attack/Name_Calling-Labeling_v001,persuasion,...,"[None, None, None]","[True, True, True]","[{""Description"": ""Technique used: Yes — Name C...",[Technique used: Yes — Name Calling / Labeling...,"[1, 1, 1]","[None, None, None]","[None, None, None]","[None, None, None]",1,1
2,pl_current_affairs_09_01_2025_n03.txt,263,872,PL,"(Poseł Władysław Dajczak: Ha, ha, ha! To prawd...",True,"[Conversation_Killer, Appeal_to_Hypocrisy, Loa...",LLAMA_32_3B/attack/Name_Calling-Labeling_v001,attack/Name_Calling-Labeling_v001,persuasion,...,"[None, None, None]","[True, True, True]","[```\n{\n ""Description"": ""Technique used: Yes...",[Technique used: Yes — Name Calling / Labeling...,"[1, 1, 1]","[None, None, None]","[None, None, None]","[None, None, None]",1,0
3,pl_current_affairs_09_01_2025_n03.txt,874,1322,PL,Kolejne pytanie. Na stronie Wód Polskich pojaw...,True,[Repetition],LLAMA_32_3B/attack/Name_Calling-Labeling_v001,attack/Name_Calling-Labeling_v001,persuasion,...,"[None, None, None]","[True, True, True]","[{""Description"": ""Technique used: Yes — Name C...",[Technique used: Yes — Name Calling / Labeling...,"[1, 0, 0]","[None, None, None]","[None, None, None]","[None, None, None]",0,0
4,pl_current_affairs_09_01_2025_n03.txt,1324,1465,PL,Jeszcze tylko jedna rzecz. Nawet politycy Lewi...,True,[Appeal_to_Popularity],LLAMA_32_3B/attack/Name_Calling-Labeling_v001,attack/Name_Calling-Labeling_v001,persuasion,...,"[None, None, None]","[True, True, True]","[{""Description"": ""Technique used: Yes — Name C...",[Technique used: Yes — Name Calling / Labeling...,"[1, 1, 1]","[None, None, None]","[None, None, None]","[None, None, None]",1,0
5,pl_current_affairs_09_01_2025_n03.txt,1467,1528,PL,Wicemarszałek Włodzimierz Czarzasty:\nDziękuję...,False,[],LLAMA_32_3B/attack/Name_Calling-Labeling_v001,attack/Name_Calling-Labeling_v001,persuasion,...,"[None, None, None]","[True, True, True]","[{""Description"": ""Technique used: Yes — Name C...",[Technique used: Yes — Name Calling / Labeling...,"[1, 0, 0]","[None, None, None]","[None, None, None]","[None, None, None]",0,0
6,pl_current_affairs_09_01_2025_n03.txt,1530,1806,PL,Sekretarz Stanu\nw Ministerstwie Klimatu i Śro...,False,[],LLAMA_32_3B/attack/Name_Calling-Labeling_v001,attack/Name_Calling-Labeling_v001,persuasion,...,"[None, None, None]","[True, True, True]","[{""Description"": ""Technique used: No — Name Ca...",[Technique used: No — Name Calling / Labeling ...,"[0, 0, 1]","[None, None, None]","[None, None, None]","[None, None, None]",0,0
7,pl_current_affairs_09_01_2025_n03.txt,1808,1984,PL,(Poseł Anna Gembicka: Decyzja została zmienion...,True,[Repetition],LLAMA_32_3B/attack/Name_Calling-Labeling_v001,attack/Name_Calling-Labeling_v001,persuasion,...,"[None, None, None]","[True, True, True]","[```\n{\n ""Description"": ""Technique used: Yes...",[Technique used: Yes — Name Calling / Labeling...,"[1, 1, 1]","[None, None, None]","[None, None, None]","[None, None, None]",1,0
8,pl_current_affairs_09_01_2025_n03.txt,1986,2437,PL,Sąd bazował na uzasadnieniu o niedostatecznym\...,True,[Repetition],LLAMA_32_3B/attack/Name_Calling-Labeling_v001,attack/Name_Calling-Labeling_v001,persuasion,...,"[None, No

In [ ]:
from factue.methods.llm_calls import load_template_parts

In [ ]:
job = "persuasion"
step = "detect"
prompt_id = "manipulative/Loaded_Language_v001.yaml"

template_parts = load_template_parts(job, step, prompt_id)

In [ ]:
json.dumps(payload)

In [ ]:
from jsonschema import validate, ValidationError
import json
import re
import ast
import demjson3

def extract_json_from_payload(payload):
    # Extract <think> content
    think_match = re.search(r"<think>(.*?)</think>", payload, flags=re.DOTALL | re.IGNORECASE)
    think_content = think_match.group(1).strip() if think_match else None

    # Remove <think>...</think> block
    payload_wo_think = re.sub(r"<think>.*?</think>", "", payload, flags=re.DOTALL | re.IGNORECASE)

    # Remove Markdown code fences and XML-like tags
    cleaned = re.sub(r"```json|```|<[^>]+>", "", payload_wo_think, flags=re.IGNORECASE).strip()

    # Find the first valid JSON object using balanced braces
    brace_stack = []
    start_idx = None
    for i, char in enumerate(cleaned):
        if char == '{':
            if not brace_stack:
                start_idx = i
            brace_stack.append('{')
        elif char == '}':
            if brace_stack:
                brace_stack.pop()
                if not brace_stack:
                    json_str = cleaned[start_idx:i+1]
                    # Fix trailing commas before } or ]
                    json_str = re.sub(r',\s*([}\]])', r'\1', json_str)
                    extra_content = cleaned[:start_idx] + cleaned[i+1:]
                    return json_str.strip(), think_content, extra_content.strip() or None
    return None, think_content, cleaned.strip() or None

def normalize_keys(data):
    return {k.lower(): v for k, v in data.items()}

def coerce_boolean_fields(data, boolean_fields):
    illegal_values = {}
    for field in boolean_fields:
        if field in data:
            original_value = data[field]
            if isinstance(original_value, str):
                val = original_value.strip().lower()
                if val in ["true", "yes", "1"]:
                    data[field] = True
                elif val in ["false", "no", "0"]:
                    data[field] = False
                else:
                    data[field] = False
                    illegal_values[field] = original_value
            elif isinstance(original_value, (int, float)):
                data[field] = bool(original_value)
            elif isinstance(original_value, bool):
                pass
            else:
                data[field] = False
                illegal_values[field] = original_value
    return illegal_values

def validate_response(payload):
    raw_json, think_content, extra_content = extract_json_from_payload(payload)

    schema_properties = {k.lower(): v for k, v in schema["properties"].items()}
    required_fields = [k.lower() for k in schema["required"]]
    boolean_fields = [k for k, v in schema_properties.items() if v.get("type") == "boolean"]

    result = {
        **{key: None for key in schema_properties},
        "extra_properties": None,
        "think_content": think_content,
        "extra_content": extra_content,
        "illegal_value": {},
        "raw": payload,
        "status": False
    }

    if not raw_json:
        return result

    try:
        data = json.loads(raw_json)
    except json.JSONDecodeError:
        try:
            data = ast.literal_eval(raw_json)
        except Exception:
            try:
                data = demjson3.decode(raw_json)
            except Exception:
                return result

    data = normalize_keys(data)
    illegal_values = coerce_boolean_fields(data, boolean_fields)

    lower_schema = {
        "type": schema["type"],
        "properties": schema_properties,
        "required": required_fields,
        "additionalProperties": schema.get("additionalProperties", True)
    }

    try:
        validate(instance=data, schema=lower_schema)
    except ValidationError:
        return result

    allowed_keys = set(schema_properties.keys())
    actual_keys = set(data.keys())
    extra_keys = actual_keys - allowed_keys

    extra_properties = {key: data.pop(key) for key in extra_keys} if extra_keys else None
    if extra_properties:
        result["extra_properties"] = json.dumps(extra_properties)

    for key in schema_properties:
        result[key] = data.get(key)

    result["illegal_value"] = {k: illegal_values.get(k, "ok") for k in boolean_fields}
    result["status"] = all(field in data for field in required_fields)
    return result

schema = {
    "type": "object",
    "properties": {
        "Description": {"type": "string"},
        "Verdict": {"type": "boolean"}
    },
    "required": ["Description", "Verdict"],
    "additionalProperties": True
}

payload = '<think>xyz</think>```json {"Desciption": "abc", "Verdict": "yes", "Other": "123"} ``` some trailing notes.'
payload = "{{\n  \"Description\": \"Technique used: No \u2014 Name Calling / Labeling is not present.\",\n  \"Verdict\": false,\n}}"
payload = "{\n  \"Description\": \"Technique used: No \u2014 Name Calling / Labeling is not present.\",\n  \"Verdict\": false,\n}"
# payload = '{\n \"Description": "Technique\u2014","Verdict": false}'
# payload = '{"Description": "abc", "Verdict": "yes", "Other": "123"}.'

result = validate_response(payload)
result


In [ ]:
import demjson3

In [ ]:
payload = "{\n  \"Description\": \"Technique used: No \u2014 Name Calling / Labeling is not present.\",\n  \"Verdict\": false,\n}"

demjson3.decode(payload)

In [ ]:
from jsonschema import validate, ValidationError
import json

def validate_response(payload):
    data = json.loads(payload)
    try:
        validate(instance=data, schema=schema)
        print("Payload is valid ✅")
    except ValidationError as e:
        print("Invalid payload ❌:", e.message)

schema = {
    "type": "object",
    "properties": {
        "Desciption": {"type": "string"},
        "Verdict": {"type": "boolean"}
    },
    "required": ["Desciption", "Verdict"],
    "additionalProperties": True
}

payload = '{"Desciption": "abc", "Verdict": true, "Other": "123"}'

validate_response(payload)

# call


In [8]:
from factue.methods.llm_calls import make_call
from factue.methods.llm_langchain.llm import Llm
from factue.utils.types import ModelMode, ModelProvider, ModelName, Job
import pandas as pd

In [3]:
llm_params = {
            "model_name": ModelName.LLAMA_32_3B,
            "model_provider": ModelProvider.OLLAMA,
            "model_mode": ModelMode.CHAT,
            "seed": -1,
        }
llm = Llm(**llm_params)

seed:  -1


In [ ]:
text = """Poseł Anna Gembicka:
Bardzo dziękuję.
Panie Marszałku! Wysoka Izbo! Pani Minister!
Przypomnę pani kilka faktów, bo widzę, że faktycznie w wielu obszarach jest pani zielona.
"""

In [19]:
df = pd.DataFrame({
    'txt': [
        text,
        text
    ]
})
df

,txt
0,Poseł Anna Gembicka:\nBardzo dziękuję.\nPanie ...
1,Poseł Anna Gembicka:\nBardzo dziękuję.\nPanie ...


In [23]:
result = df.apply(lambda row: make_call(
                llm=llm,
                job=Job.PERSUASION.value,
                step='detect',
                prompt_id="attack/Name_Calling-Labeling_v001",
                variables={"text": row['txt']},
                max_iterations=2,)
            ,axis=1,)

2025-05-06 16:59:27,940 [INFO] httpx: HTTP Request: POST http://0.0.0.0:5000/api/chat "HTTP/1.1 200 OK"
2025-05-06 16:59:30,705 [INFO] httpx: HTTP Request: POST http://0.0.0.0:5000/api/chat "HTTP/1.1 200 OK"
2025-05-06 16:59:32,889 [INFO] httpx: HTTP Request: POST http://0.0.0.0:5000/api/chat "HTTP/1.1 200 OK"
2025-05-06 16:59:34,454 [INFO] httpx: HTTP Request: POST http://0.0.0.0:5000/api/chat "HTTP/1.1 200 OK"


In [24]:
result.tolist()

[[{'raw': '```\n{\n  "Description": "Technique used: Yes — Name Calling / Labeling is present.The phrase „Panie Marszałku! Wysoka Izbo! Pani Minister“ is a typical example of the Name Calling - Labeling technique: Using titles such as „Panie“ and „Pani“ are formal, but in this context, they can be perceived as condescending or patronizing. The phrase „Wysoka Izbo“ is another label that positions the speaker above others in terms of power and authority. The combination of these terms in a nominal group (without presenting evidence or argumentation) reduces the address to a hierarchical identity. Rather than objectively addressing the recipient, the sentence uses loaded labels to provoke emotional judgment in the reader and frame the event and people involved negatively from the outset.",\n  "Verdict": true\n}\n```',
   'status': True},
  {'raw': '```\n{\n  "Description": "Technique used: Yes — Name Calling / Labeling is present.The phrase \\"Panie Marszałku! Wysoka Izbo! Pani Minister!\

In [5]:
result = make_call(
                llm=llm,
                job=Job.PERSUASION.value,
                step='detect',
                prompt_id="attack/Name_Calling-Labeling_v001",
                variables={"text": text, "text_lang": "pl"},
                max_iterations=2,
            )

2025-05-06 16:51:45,447 [INFO] httpx: HTTP Request: POST http://0.0.0.0:5000/api/chat "HTTP/1.1 200 OK"
2025-05-06 16:51:46,798 [INFO] httpx: HTTP Request: POST http://0.0.0.0:5000/api/chat "HTTP/1.1 200 OK"


In [11]:
pd.DataFrame(result.tolist())

AttributeError: 'list' object has no attribute 'tolist'

In [6]:
result

[{'raw': '```\n{\n  "Description": "Technique used: Yes — Name Calling / Labeling is present.The terms \'Panie Marszałku\' (Mr. Marshal), \'Wysoka Izbo\' (Honorable Chamber), and \'Pani Minister\' (Ms. Minister) are formal titles used to address public figures in a respectful manner. However, the phrase \'Pani zielona\' (Ms. Green) is an example of Name Calling / Labeling: it assigns a pejorative label (\'zielona\' meaning \'green\' in Polish), implying that the politician is somehow deceitful or dishonest. This is used to provoke emotional judgment and frame the event and people involved negatively from the outset.",\n  "Verdict": true\n}\n```',
  'status': True},
 {'raw': '{\n  "Description": "Technique used: Yes — Name Calling / Labeling is present.The use of titles such as \'Panie Marszałku\' (Mr. Marshal), \'Wysoka Izbo\' (Honorable Assembly), and \'Pani Minister\' (Ms. Minister) can be seen as pejorative or ironic, implying a sense of condescension or superiority. Additionally, t